## **Movies Recommendation: Matrix Factorized Collaborative Filtering**

### **Import necessary library**

In [1]:
import pandas as pd 
import numpy as np 
import math
from collaborative_filtering import MatrixFactorizationCF, build_utility_matrix


In [2]:
# import pandas as pd

# part_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]

# # Read the first file
# data = pd.read_csv('resources/ratings/users_ratings_part1.csv', header=None)

# # Loop through other files
# for id in part_ids:
#     file_path = f'resources/ratings/users_ratings_part{id}.csv'
#     try:
#         tmp = pd.read_csv(file_path, on_bad_lines="skip", header=None)  # Skip problematic rows
#         data = pd.concat([data, tmp], ignore_index=True)
#     except Exception as e:
#         print(f"Error reading file {file_path}: {e}")

# data.columns =['UserID', 'MovieID', 'Rating']

data = pd.read_csv('resources/ratings/users_ratings_part1.csv', header=None)
data.columns =['UserID', 'MovieID', 'Rating']

In [3]:
data = data.sort_values(by=['UserID', 'MovieID'])
data = data.drop_duplicates()
data = data.reset_index()
data = data.drop(columns=['index'])

In [4]:
data.head()

,UserID,MovieID,Rating
0,9423,27977,8
1,9423,31381,9
2,9423,32138,10
3,9423,33467,10
4,9423,38650,10


In [5]:
data.tail()

,UserID,MovieID,Rating
54972,186180127,468569,10
54973,186180127,1255953,10
54974,186180127,9362722,10
54975,186180127,11329280,10
54976,186180127,15239678,10


In [6]:
# Count the number of interactions for each movie
movie_counts =data['MovieID'].value_counts()

# Filter out movies appear < 10 times 
movies_to_keep = movie_counts[movie_counts >= 10].index
data = data[data['MovieID'].isin(movies_to_keep)]

# Count the number of interactions for each user
user_counts = data['UserID'].value_counts()

# Filter out users appear < 10 times 
users_to_keep = user_counts[user_counts >= 10].index 
data = data[data['UserID'].isin(users_to_keep)]

In [7]:
num_users = data.UserID.unique().shape[0]
num_movies = data.MovieID.unique().shape[0]
num_users, num_movies

(267, 910)

### **Split data**

In [8]:
def split_data(df, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    
    # Preprocessing Step: Remove movies with frequency = 1
    movie_counts = df['MovieID'].value_counts()
    movies_to_keep = movie_counts[movie_counts > 1].index
    df = df[df['MovieID'].isin(movies_to_keep)].reset_index(drop=True)
    
    # Initialize sets to keep track of test indices
    test_indices = set()
    
    # Ensure each user has at least one entry in test set
    for user_id, group in df.groupby('UserID'):
        if len(group) >= 1:
            test_count = max(1, int(len(group) * test_size))
            test_sample = np.random.choice(group.index, size=test_count, replace=False)
            test_indices.update(test_sample)
    
    # Ensure each movie has at least one entry in test set
    for movie_id, group in df.groupby('MovieID'):
        # Find indices not already in test set
        remaining_indices = list(set(group.index) - test_indices)
        if remaining_indices:
            test_sample = np.random.choice(remaining_indices, size=1, replace=False)
            test_indices.update(test_sample)
    
    # Create test and train datasets
    test_df = df.loc[list(test_indices)].reset_index(drop=True)
    train_df = df.drop(list(test_indices)).reset_index(drop=True)
    
    # Optional: Verify that both train and test have the same users and movies
    common_users = set(train_df['UserID']).intersection(set(test_df['UserID']))
    common_movies = set(train_df['MovieID']).intersection(set(test_df['MovieID']))
    
    train_df = train_df[train_df['UserID'].isin(common_users) & train_df['MovieID'].isin(common_movies)].reset_index(drop=True)
    test_df = test_df[test_df['UserID'].isin(common_users) & test_df['MovieID'].isin(common_movies)].reset_index(drop=True)
    
    return train_df, test_df

# Example usage:
# Assuming your dataframe is named `df` and has columns 'UserID' and 'MovieID'
# train, test = split_data(df, test_size=0.2, random_state=42)


In [9]:
train_data, test_data = split_data(data)

In [10]:
train_data.head()

,UserID,MovieID,Rating
0,9423,27977,8
1,9423,31381,9
2,9423,32138,10
3,9423,33467,10
4,9423,38650,10


In [11]:
train_data.tail()

,UserID,MovieID,Rating
28376,186180127,172495,10
28377,186180127,407887,10
28378,186180127,468569,10
28379,186180127,9362722,10
28380,186180127,15239678,10


In [12]:
num_users = train_data.UserID.unique().shape[0]
num_movies = train_data.MovieID.unique().shape[0]
num_users, num_movies

(267, 910)

In [13]:
test_data.head()

,UserID,MovieID,Rating
0,106825954,365748,1
1,9423,53125,9
2,9423,52618,8
3,9423,53291,7
4,106825954,372784,8


In [14]:
test_data.tail()

,UserID,MovieID,Rating
8134,106825954,209144,9
8135,106825954,234215,7
8136,106825954,245429,9
8137,106825954,268380,8
8138,106825954,290334,7


In [15]:
num_users = test_data.UserID.unique().shape[0]
num_movies = test_data.MovieID.unique().shape[0]
num_users, num_movies

(267, 910)

In [16]:
movies = train_data.MovieID.drop_duplicates()
movies = pd.DataFrame(movies, columns=['MovieID'])
movies = movies.sort_values('MovieID', ascending=True)
movies = movies[['MovieID']]
movies = movies.reset_index()

In [17]:
users = train_data.UserID.drop_duplicates()
users = pd.DataFrame(users, columns=['UserID'])
users = users.sort_values('UserID', ascending=True)
users = users[['UserID']]
users = users.reset_index()

### **Build Utility Matrix**

In [18]:
utility_matrix = build_utility_matrix(train_data)
utility_matrix

array([[ 8.,  9., 10., ...,  0.,  0.,  0.],
       [10.,  7.,  9., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  7.,  8.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.]])

## **User-based MF**

In [19]:
UUMF = MatrixFactorizationCF(R=utility_matrix, K=45, learning_rate=0.005, epochs=50, regularization=0.1)
UUMF.train()

Epoch: 1 - RMSE: 1.8365
Epoch: 2 - RMSE: 1.7349
Epoch: 3 - RMSE: 1.6944
Epoch: 4 - RMSE: 1.6728
Epoch: 5 - RMSE: 1.6591
Epoch: 6 - RMSE: 1.6494
Epoch: 7 - RMSE: 1.6423
Epoch: 8 - RMSE: 1.6367
Epoch: 9 - RMSE: 1.6321
Epoch: 10 - RMSE: 1.6282
Epoch: 11 - RMSE: 1.6247
Epoch: 12 - RMSE: 1.6217
Epoch: 13 - RMSE: 1.6186
Epoch: 14 - RMSE: 1.6157
Epoch: 15 - RMSE: 1.6128
Epoch: 16 - RMSE: 1.6094
Epoch: 17 - RMSE: 1.6059
Epoch: 18 - RMSE: 1.6017
Epoch: 19 - RMSE: 1.5967
Epoch: 20 - RMSE: 1.5910
Epoch: 21 - RMSE: 1.5844
Epoch: 22 - RMSE: 1.5766
Epoch: 23 - RMSE: 1.5678
Epoch: 24 - RMSE: 1.5583
Epoch: 25 - RMSE: 1.5480
Epoch: 26 - RMSE: 1.5370
Epoch: 27 - RMSE: 1.5255
Epoch: 28 - RMSE: 1.5136
Epoch: 29 - RMSE: 1.5016
Epoch: 30 - RMSE: 1.4895
Epoch: 31 - RMSE: 1.4775
Epoch: 32 - RMSE: 1.4657
Epoch: 33 - RMSE: 1.4540
Epoch: 34 - RMSE: 1.4425
Epoch: 35 - RMSE: 1.4311
Epoch: 36 - RMSE: 1.4200
Epoch: 37 - RMSE: 1.4090
Epoch: 38 - RMSE: 1.3982
Epoch: 39 - RMSE: 1.3877
Epoch: 40 - RMSE: 1.3774
Epoch: 41

In [20]:
predicted_R = UUMF.full_prediction()

In [21]:
recommendation = UUMF.recommend(id=9423, predicted_R=predicted_R, movies=movies, users=users, top_n=5)
print('Recommend movies for user 9423 and predicted ratings:')
recommendation

Recommend movies for user 9423 and predicted ratings:


[(47478, 9), (60196, 9), (110912, 9), (405094, 9), (12361974, 9)]

In [22]:
# UUMF.print_recommendation(predicted_R=predicted_R, movies=movies, users=users)

## **Item-based MF**

In [23]:
IIMF = MatrixFactorizationCF(R=utility_matrix, K=45, learning_rate=0.005, epochs=50, regularization=0.1, uu_mf=False)
IIMF.train()

Epoch: 1 - RMSE: 1.8362
Epoch: 2 - RMSE: 1.7347
Epoch: 3 - RMSE: 1.6943
Epoch: 4 - RMSE: 1.6725
Epoch: 5 - RMSE: 1.6588
Epoch: 6 - RMSE: 1.6493
Epoch: 7 - RMSE: 1.6422
Epoch: 8 - RMSE: 1.6365
Epoch: 9 - RMSE: 1.6320
Epoch: 10 - RMSE: 1.6281
Epoch: 11 - RMSE: 1.6245
Epoch: 12 - RMSE: 1.6213
Epoch: 13 - RMSE: 1.6182
Epoch: 14 - RMSE: 1.6152
Epoch: 15 - RMSE: 1.6119
Epoch: 16 - RMSE: 1.6083
Epoch: 17 - RMSE: 1.6041
Epoch: 18 - RMSE: 1.5995
Epoch: 19 - RMSE: 1.5939
Epoch: 20 - RMSE: 1.5877
Epoch: 21 - RMSE: 1.5804
Epoch: 22 - RMSE: 1.5721
Epoch: 23 - RMSE: 1.5631
Epoch: 24 - RMSE: 1.5532
Epoch: 25 - RMSE: 1.5429
Epoch: 26 - RMSE: 1.5320
Epoch: 27 - RMSE: 1.5208
Epoch: 28 - RMSE: 1.5092
Epoch: 29 - RMSE: 1.4976
Epoch: 30 - RMSE: 1.4859
Epoch: 31 - RMSE: 1.4741
Epoch: 32 - RMSE: 1.4626
Epoch: 33 - RMSE: 1.4511
Epoch: 34 - RMSE: 1.4399
Epoch: 35 - RMSE: 1.4287
Epoch: 36 - RMSE: 1.4177
Epoch: 37 - RMSE: 1.4070
Epoch: 38 - RMSE: 1.3964
Epoch: 39 - RMSE: 1.3860
Epoch: 40 - RMSE: 1.3761
Epoch: 41

In [24]:
predicted_R = IIMF.full_prediction()

In [25]:
predicted_R[predicted_R > 10]

array([], dtype=float64)

In [26]:
recommendation = IIMF.recommend(id=27977, predicted_R=predicted_R, movies=movies, users=users, top_n=5)
print('Recommend users for movie 27977 and predicted ratings:')
recommendation

Recommend users for movie 27977 and predicted ratings:


[(1257208, 10), (26355627, 10), (60255714, 10), (61541609, 10), (65718654, 10)]

In [27]:
# IIMF.print_recommendation(predicted_R=predicted_R, movies=movies, users=users)